In [6]:
# Fast-forward >>>> just clean the data
%run data_cleaning.py

/Users/masonphung/Desktop/data science/data_projects/macquarie_dataviz24/macquarie_dataviz24/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(


In [37]:
# import libraries
import pandas as pd
import numpy as np
import datetime

# Load raw data
df = pd.read_excel('dataset/raw_data/public_emdat_20240923.xlsx')

In [46]:
# Choose desired variables
data = df[["DisNo.", "Disaster Type", "ISO", "Country", "Subregion", "Region", "Start Year", "Start Month", 
             "Total Deaths", "Total Affected", "Total Damage, Adjusted ('000 US$)", "Last Update"]]

# Rename the columns
data.rename(
    columns = {
        "DisNo.": "id", "Disaster Type": "type", "ISO": "iso", "Country": "country", 
        "Subregion": "subregion", "Region": "region", "Start Year": "year", "Start Month": "month", 
        "Total Deaths": "total_deaths", "Total Affected": "total_affected", 
        "Total Damage, Adjusted ('000 US$)": "total_damage", "Last Update": "last_update"
    },
    inplace = True
)


# Drop non-natural disaster and disasters that only have 1 (not enough) observations
remove_vars = ['Animal incident', 'Epidemic', 'Impact', 'Infestation']
data = data[~data['type'].isin(remove_vars)]

/var/folders/v2/whcxjrn109q5520vvfscjz_00000gn/T/ipykernel_88506/748582280.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(


In [47]:
# Remove observations with null Month
data = data[~data['month'].isnull()]

# Convert null values in total damage, deaths and affected to '-1'
for i in ['total_damage', 'total_deaths', 'total_affected']:
    data[i] = data[i].replace(np.nan, -1)


In [50]:
# Add a yyyy/mm variable
data['date'] = data.apply(lambda row: f"{int(row['year'])}/{int(row['month']):02d}", axis=1)

# Define the mapping for disaster types
disaster_mapping = {
    'Flood': 'Flood',
    'Glacial lake outburst flood': 'Flood',
    'Mass movement (wet)': 'Mass movement',
    'Mass movement (dry)': 'Mass movement',
    'Drought': 'Drought',
    'Extreme temperature': 'Extreme temperature',
    'Volcanic activity': 'Volcanic activity',
    'Storm': 'Storm',
    'Wildfire': 'Wildfire',
    'Earthquake': 'Earthquake'
}

# Group the disaster types with defined mapping
data['type'] = data['type'].map(disaster_mapping)

# Now map each disaster type to an integer code
type_codes = {
    'Drought': 1,
    'Flood': 2,
    'Extreme temperature': 3,
    'Volcanic activity': 4,
    'Storm': 5,
    'Wildfire': 6,
    'Earthquake': 7,
    'Mass movement': 8
}

# Create a new column 'type_code' by mapping the disaster types to their integer codes
data['type_code'] = data['type'].map(type_codes)

# Recheck
data['type'].unique()


array(['Drought', 'Flood', 'Extreme temperature', 'Volcanic activity',
       'Storm', 'Wildfire', 'Earthquake', 'Mass movement'], dtype=object)

In [51]:
# Take a look at the data in general
data

,id,type,iso,country,subregion,region,year,month,total_deaths,total_affected,total_damage,last_update,date,type_code
0,1999-9388-DJI,Drought,DJI,Djibouti,Sub-Saharan Africa,Africa,2001,6.0,-1.0,100000.0,-1.0,2023-09-25,2001/06,1
1,1999-9388-SDN,Drought,SDN,Sudan,Northern Africa,Africa,2000,1.0,-1.0,2000000.0,-1.0,2023-09-25,2000/01,1
2,1999-9388-SOM,Drought,SOM,Somalia,Sub-Saharan Africa,Africa,2000,1.0,21.0,1200000.0,-1.0,2023-09-25,2000/01,1
3,2000-0002-AGO,Flood,AGO,Angola,Sub-Saharan Africa,Africa,2000,1.0,31.0,70000.0,17695.0,2023-09-25,2000/01,2
4,2000-0003-BGD,Extreme temperature,BGD,Bangladesh,Southern Asia,Asia,2000,1.0,49.0,-1.0,-1.0,2023-09-25,2000/01,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10149,2024-9197-ZWE,Drought,ZWE,Zimbabwe,Sub-Saharan Africa,Africa,2024,4.0,-1.0,7600000.0,-1.0,2024-08-12,2024/04,1
10150,2024-9277-VNM,Drought,VNM,Viet Nam,South-eastern Asia,Asia,2024,1.0,-1.0,266000.0,-1.0,2024-08-12,2024/01,1
10151,2024-9338-NAM,Drought,NAM,Namibia,Sub-Saharan Africa,Africa,2024,5.0,-1.0,1655000.0,-1.0,2024-08-12,2024/05,1
10152,2024-9381-GRD,Drought,GRD,Grenada,Latin America and the Caribbean,Americas,2024,5.0,-1.0,100000.0,-1.0,2024-08-12,2024/05,1


In [10]:
# Define a function to report missing data
def report_missing(df):
    """
    Create a dataframe, then calculate the number of null and blank value.
    
    Parameter:
    df (DataFrame)
        The dataframe used to check for missing values
    
    Return:
    completed_report (DataFrame)
        The report table including the number of null, blank values and their percentage in total
    """
    # Total observation count
    total_obs = df.shape[0]
    # Create a dataframe
    missing = pd.DataFrame()
    # Total nulls
    missing['null_count'] = df.isnull().sum()
    # Total blank value
    missing['blank_count'] = [df[df[c].astype(str) == ""][c].count() for c in df.columns]
    # Total missing value
    missing['total_missing'] = missing.sum(axis = 1)
    # Report missing percentage
    missing['null_percent'] = round(100* (missing['null_count']/ total_obs), 2)
    missing['blank_percent'] = round(100* (missing['blank_count']/ total_obs), 2)
    missing['total_missing_percent'] = round(100* (missing['total_missing']/ total_obs), 2)
    
    completed_report = missing.sort_values(
        by = 'total_missing_percent',
        ascending = False
    )
    return completed_report

In [52]:
report_missing(data)

,null_count,blank_count,total_missing,null_percent,blank_percent,total_missing_percent
id,0,0,0,0.0,0.0,0.0
type,0,0,0,0.0,0.0,0.0
iso,0,0,0,0.0,0.0,0.0
country,0,0,0,0.0,0.0,0.0
subregion,0,0,0,0.0,0.0,0.0
region,0,0,0,0.0,0.0,0.0
year,0,0,0,0.0,0.0,0.0
month,0,0,0,0.0,0.0,0.0
total_deaths,0,0,0,0.0,0.0,0.0
total_affected,0,0,0,0.0,0.0,0.0
